In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.optimizers import Adam, RMSprop

In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.optimizers import Adam, RMSprop

# Load Data
books = pd.read_csv('Data/BX_book/BX-Books.csv', sep=';', encoding="latin-1", on_bad_lines='skip')
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']
users = pd.read_csv('Data/BX_book/BX-Users.csv', sep=';', on_bad_lines='skip', encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']
ratings = pd.read_csv('Data/BX_book/BX-Book-Ratings.csv', sep=';', on_bad_lines='skip', encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

# Merge books and ratings
data = pd.merge(ratings, books, on='ISBN')

# Create user and book id mapping
user_ids = data['userID'].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
user_encoded2user = {i: x for i, x in enumerate(user_ids)}
book_ids = data['ISBN'].unique().tolist()
book2book_encoded = {x: i for i, x in enumerate(book_ids)}
book_encoded2book = {i: x for i, x in enumerate(book_ids)}

# Map user and book ids to user and book indices
data['user'] = data['userID'].map(user2user_encoded)
data['book'] = data['ISBN'].map(book2book_encoded)

# Split data into training and testing set
train, test = train_test_split(data, test_size=0.2, random_state=42)

# Get number of users and books
num_users = len(user2user_encoded)
num_books = len(book_encoded2book)

# Set embedding dimension
embedding_size=10

# Build model
user_input = Input(shape=[1])
user_embedding = Embedding(num_users, embedding_size)(user_input)
user_vec = Flatten()(user_embedding)

book_input = Input(shape=[1])
book_embedding = Embedding(num_books, embedding_size)(book_input)
book_vec = Flatten()(book_embedding)

product = Dot(axes=1)([book_vec, user_vec])
model = Model(inputs=[user_input, book_input], outputs=product)
model.compile(loss='mean_squared_error', optimizer=Adam())

# Train model
history = model.fit(x=[train.user.values, train.book.values], y=train.bookRating.values, batch_size=64, epochs=5, verbose=1, validation_split=1)

# Evaluate model
mse = model.evaluate([test.user.values, test.book.values], test.bookRating.values)
print(f'Test MSE: {mse}')

C:\Users\dimgr\AppData\Local\Temp\ipykernel_17572\3633506929.py:9: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Data/BX_book/BX-Books.csv', sep=';', encoding="latin-1", on_bad_lines='skip')


ValueError: Training data contains 824908 samples, which is not sufficient to split it into a validation and training set as specified by `validation_split=1`. Either provide more data, or a different value for the `validation_split` argument.

In [9]:
books = pd.read_csv('Data/BX_book/BX-Books.csv', sep=';', encoding="latin-1", on_bad_lines='skip')
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']
users = pd.read_csv('Data/BX_book/BX-Users.csv', sep=';', on_bad_lines='skip', encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']
ratings = pd.read_csv('Data/BX_book/BX-Book-Ratings.csv', sep=';', on_bad_lines='skip', encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

C:\Users\dimgr\AppData\Local\Temp\ipykernel_17572\87431705.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Data/BX_book/BX-Books.csv', sep=';', encoding="latin-1", on_bad_lines='skip')


In [10]:
data = pd.merge(ratings, books, on='ISBN')

In [11]:
user_ids = data['userID'].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
user_encoded2user = {i: x for i, x in enumerate(user_ids)}
book_ids = data['ISBN'].unique().tolist()
book2book_encoded = {x: i for i, x in enumerate(book_ids)}
book_encoded2book = {i: x for i, x in enumerate(book_ids)}

In [12]:
data['user'] = data['userID'].map(user2user_encoded)
data['book'] = data['ISBN'].map(book2book_encoded)

In [27]:
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [21]:
num_users = len(user2user_encoded)
num_books = len(book_encoded2book)

In [22]:
embedding_size=10

In [31]:
user_input = Input(shape=[1])
user_embedding = Embedding(num_users, embedding_size)(user_input)
user_vec = Flatten()(user_embedding)

book_input = Input(shape=[1])
book_embedding = Embedding(num_books, embedding_size)(book_input)
book_vec = Flatten()(book_embedding)

product = Dot(axes=1)([book_vec, user_vec])
model = Model(inputs=[user_input, book_input], outputs=product)
model.compile(loss='mean_squared_error', optimizer=RMSprop())

In [38]:
history = model.fit(x=[train.user.values, train.book.values], y=train.bookRating.values, 
                    batch_size=64, epochs=200, verbose=1, 
                    validation_data=([test.user.values, test.book.values], test.bookRating.values))



Epoch 1/200
12890/12890 [==============================] - 186s 14ms/step - loss: 19.4291 - val_loss: 36.0191
Epoch 2/200
12890/12890 [==============================] - 230s 18ms/step - loss: 19.0363 - val_loss: 35.7929
Epoch 3/200
12890/12890 [==============================] - 223s 17ms/step - loss: 18.6958 - val_loss: 35.6269
Epoch 4/200
12890/12890 [==============================] - 219s 17ms/step - loss: 18.3788 - val_loss: 35.4794
Epoch 5/200
12890/12890 [==============================] - 215s 17ms/step - loss: 18.0959 - val_loss: 35.3539
Epoch 6/200
12890/12890 [==============================] - 212s 16ms/step - loss: 17.8166 - val_loss: 35.2549
Epoch 7/200
12890/12890 [==============================] - 216s 17ms/step - loss: 17.5692 - val_loss: 35.1819
Epoch 8/200
12890/12890 [==============================] - 219s 17ms/step - loss: 17.3357 - val_loss: 35.1201
Epoch 9/200
12890/12890 [==============================] - 215s 17ms/step - loss: 17.1181 - val_loss: 35.0819
Epoch 10/2

KeyboardInterrupt: 

In [ ]:

# Evaluate model
mse = model.evaluate([test.user.values, test.book.values], test.bookRating.values)
print(f'Test MSE: {mse}')

6445/6445 [==============================] - 3s 527us/step - loss: 20.1628
Test MSE: 20.16280746459961


In [ ]:
model.save('Data/BX_book/TF_boork-rec.h5')